In [ ]:
from openai import OpenAI
from key import openai_key
client = OpenAI(api_key=openai_key)


b=0
# around 30min to run one batch
batch_input_file = client.files.create(
  file=open("../../gpt4o_data/batch_request_gpt4o_"+str(b+1)+".jsonl", "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id

metadata = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "version 1.0",
    }
)

print(metadata)


Batch(id='batch_673233b412c08190abe7538db2a6e686', completion_window='24h', created_at=1731343284, endpoint='/v1/chat/completions', input_file_id='file-dYarA5tDIX8AL1IRUfi4psUF', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731429684, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'version 1.0'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


# Wait until it finished, you can run it if you are not sure whether it is finished

In [3]:
batch_id = metadata.id
metadata = client.batches.retrieve(batch_id)
batch_status = metadata.status 
print(batch_status)
output_file_id = metadata.output_file_id

file_response = client.files.content(output_file_id)
output_file_id
# print(file_response.text)

completed


'file-q8WjxbThqGmFpGDtUnBt1MjE'

In [4]:
import json 
ids = []
responses = []
for data in file_response.iter_lines():
    # print(data)
    output = json.loads(data)
    
    # print(output['custom_id'], output['response']['body']['choices'][0]['message']['content'])
    ids.append(output['custom_id'])
    responses.append(output['response']['body']['choices'][0]['message']['content'])

import pandas as pd
df = pd.DataFrame({"sm_id": ids,"responses": responses})

# Write to CSV while preserving line breaks
csv_file_path = "../../gpt4o_data/output"+str(b+1)+".csv"
df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
print(f"CSV file created at {csv_file_path}")


CSV file created at ../../gpt4o_data/output2.csv


In [5]:
read_df = pd.read_csv(csv_file_path)
read_df.head()

,sm_id,responses
0,164cx66,"(1) relation: no, related phrases if any: \n(2..."
1,164cy06,"(1) relation: no, related phrases if any: \n(2..."
2,164czru,"(1) relation: no, related phrases if any: \n(2..."
3,164d0jm,"(1) relation: no, related phrases if any: \n(..."
4,164d1lr,"(1) relation: no, related phrases if any: \n(2..."


# continue processing batch by batch

In [6]:
import json 
import os
import time
import pandas as pd
from openai import OpenAI
from key import openai_key
client = OpenAI(api_key=openai_key)

for b in range(10):

  csv_file_path = "../../gpt4o_data/output"+str(b+1)+".csv"

  # Check if the CSV file already exists
  if not os.path.exists(csv_file_path):
    print( " ----- start processing " + str(b+1) + " ----- ")

    batch_input_file = client.files.create(
      file=open("../../gpt4o_data/batch_request_gpt4o_"+str(b+1)+".jsonl", "rb"),
      purpose="batch"
    )
    batch_input_file_id = batch_input_file.id
    metadata = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
          "description": "version 1.0",
        }
    )


    batch_id = metadata.id
    batch_status = metadata.status

    # Poll every 5 minutes for batch status until completed
    while batch_status != "completed":
        time.sleep(60*5)  # Sleep for 5 minutes 
        
        # Retrieve the latest metadata to check the status
        metadata = client.batches.retrieve(batch_id)
        batch_status = metadata.status
        print(f"Batch {b + 1} status: {batch_status}")
        
        if batch_status == "completed":
            print(f"Batch {b + 1} has completed.")
            break
        

    output_file_id = metadata.output_file_id
    file_response = client.files.content(output_file_id)
    ids = []
    responses = []
    for data in file_response.iter_lines():
        output = json.loads(data)
        ids.append(output['custom_id'])
        responses.append(output['response']['body']['choices'][0]['message']['content'])

    
    df = pd.DataFrame({"sm_id": ids,"responses": responses})
    # Write to CSV while preserving line breaks
    df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
    print(f"CSV file created at {csv_file_path}")


 ----- start processing 1 ----- 
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: in_progress
Batch 1 status: finalizing
Batch 1 status: finalizing
Batch 1 status: finalizing
Batch 1 status: completed
Batch 1 has completed.
CSV file created at ../../gpt4o_data/output1.csv
 ----- start processing 3 ----- 
Batch 3 status: in_progress
Batch 3 status: in_progress
Batch 3 status: in_progress
Batch 3 status: in_progress
Batch 3 status: in_progress
Batch 3 status: in_progress
Batch 3 status: finalizing
Batch 3 status: finalizing
Batch 3 status: finalizing
Batch 3 status: completed
Batch 3 has completed.
CSV file created at ../../gpt4o_data/output3.csv
 ----- start processing 4 ----- 
Batch 4 status: in_progress
Batch 4 status: in_progress
Batch 4 status: in_progress
Batch 4 status: in_progress
Batch 4 status: finalizing
Batch 4 status: finalizing
Batch 4 statu

In [ ]:
# # need to rerun 30 sololy
# import json 
# import os
# import time
# import pandas as pd
# from openai import OpenAI
# from key import openai_key
# client = OpenAI(api_key=openai_key)

# b=29

# csv_file_path = "../../gpt4o_data/output"+str(b+1)+".csv"

# # Check if the CSV file already exists
# if not os.path.exists(csv_file_path):
#     print( " ----- start processing " + str(b+1) + " ----- ")

# batch_input_file = client.files.create(
#     file=open("../../data/batch_request_gpt4omini_"+str(b+1)+".jsonl", "rb"),
#     purpose="batch"
# )
# batch_input_file_id = batch_input_file.id
# metadata = client.batches.create(
#     input_file_id=batch_input_file_id,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#         "description": "version 1.0",
#     }
# )


# batch_id = metadata.id
# batch_status = metadata.status

# # Poll every 5 minutes for batch status until completed
# while batch_status != "completed":
#     time.sleep(60*5)  # Sleep for 2 minutes 
    
#     # Retrieve the latest metadata to check the status
#     metadata = client.batches.retrieve(batch_id)
#     batch_status = metadata.status
#     print(f"Batch {b + 1} status: {batch_status}")
    
#     if batch_status == "completed":
#         print(f"Batch {b + 1} has completed.")
#         break
    

# output_file_id = metadata.output_file_id
# file_response = client.files.content(output_file_id)
# ids = []
# responses = []
# for data in file_response.iter_lines():
#     output = json.loads(data)
#     ids.append(output['custom_id'])
#     responses.append(output['response']['body']['choices'][0]['message']['content'])


# df = pd.DataFrame({"sm_id": ids,"responses": responses})
# # Write to CSV while preserving line breaks
# df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
# print(f"CSV file created at {csv_file_path}")


 ----- start processing 30 ----- 
Batch 30 status: in_progress
Batch 30 status: in_progress
Batch 30 status: in_progress
Batch 30 status: in_progress
Batch 30 status: finalizing
Batch 30 status: completed
Batch 30 has completed.
CSV file created at ../../gpt_data/output30.csv


In [ ]:
import pandas as pd
import json

# Load the CSV file
output6 = pd.read_csv("../../gpt4o_data/output6.csv")
output6_ids = set(output6['sm_id'])  # Assuming 'sm_id' is the column name in output6.csv

# Load the JSONL file and filter for unprocessed entries
unprocessed_entries = []
entry_count = 0
with open("../../gpt4o_data/batch_request_gpt4o_6.jsonl", "r") as f:
    for line in f:
        entry = json.loads(line)
        if entry['custom_id'] not in output6_ids:  # Keep only unprocessed entries
            unprocessed_entries.append(entry)
            entry_count += 1

print(entry_count)

# Save the unprocessed entries to a new JSONL file
with open("../../gpt4o_data/batch_request_gpt4o_6_unprocess.jsonl", "w") as f:
    for entry in unprocessed_entries:
        f.write(json.dumps(entry) + "\n")

print("Unprocessed entries have been saved to 'batch_request_gpt4o_6_unprocess.jsonl'")

1520


In [16]:

b = 10
csv_file_path = "../../gpt4o_data/output"+str(b+1)+".csv"

# Check if the CSV file already exists
if not os.path.exists(csv_file_path):
    print( " ----- start processing " + str(b+1) + " ----- ")

batch_input_file = client.files.create(
    file=open("../../gpt4o_data/batch_request_gpt4o_6_unprocess.jsonl", "rb"),
    purpose="batch"
)
batch_input_file_id = batch_input_file.id
metadata = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "version 1.0",
    }
)


batch_id = metadata.id
batch_status = metadata.status

# Poll every 5 minutes for batch status until completed
while batch_status != "completed":
    time.sleep(60*5)  # Sleep for 5 minutes 
    
    # Retrieve the latest metadata to check the status
    metadata = client.batches.retrieve(batch_id)
    batch_status = metadata.status
    print(f"Batch {b + 1} status: {batch_status}")
    
    if batch_status == "completed":
        print(f"Batch {b + 1} has completed.")
        break
    

output_file_id = metadata.output_file_id
file_response = client.files.content(output_file_id)
ids = []
responses = []
for data in file_response.iter_lines():
    output = json.loads(data)
    ids.append(output['custom_id'])
    responses.append(output['response']['body']['choices'][0]['message']['content'])


df = pd.DataFrame({"sm_id": ids,"responses": responses})
# Write to CSV while preserving line breaks
df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
print(f"CSV file created at {csv_file_path}")

 ----- start processing 11 ----- 
Batch 11 status: in_progress
Batch 11 status: in_progress
Batch 11 status: in_progress
Batch 11 status: in_progress
Batch 11 status: finalizing
Batch 11 status: completed
Batch 11 has completed.
CSV file created at ../../gpt4o_data/output11.csv
